This program is free software; you can redistribute it and/or modify it under the terms of the GNU General Public License, version 2, as published by the Free Software Foundation.

This program is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for more details.

Author: Alexander Young

### This script uses sea floor age maps to mask global dynamic topography girds and calculate area weighted mean continental and oceanic air-loaded dynamic topography

##### Input:
sea floor age grid e.g. agegrid_masked_0.nc

dynamic topography mantle frame grid e.g. C1DT_MantleFrameGrid0.grd

##### Output:
time-dependent area weighted mean DT file for oceans and continents e.g. C1_mean_TopoCheck_conts_age_grid_AL.dat


In [10]:
import sys
import os
import os.path
import subprocess
import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib import gridspec
import scipy.stats as ss
import glob

# allow plots to appear within the notebook
% matplotlib inline

# Define the time range.
min_time = 0
max_time = 620
time_step = 20

# define geodynamic model
model='C3'

# path to DT grid
DTgridDir="/Users/ajy321/PhD_work/SeaLevel/SeaLevel_MS/GPlatesFiles/%s/DynamicTopography/" %(model)
# DTgridDir="/Users/ajy321/PhD_work/SeaLevel/XianzhiModel/gcm11/check/"

# path to age grid
AGEgridDir="/Users/ajy321/PhD_work/SeaLevel/EPSL_r3_calcs_05072021/AgeGrids/Mask/"
# AGEgridDir="/Users/ajy321/PhD_work/SeaLevel/XianzhiModel/SeafloorAgegrids/Mask/"

# path to output - make if it doesnt exist
dtDir="/Users/ajy321/PhD_work/SeaLevel/EPSL_r3_calcs_05072021/DTModeling"
if not os.path.exists(dtDir):
    os.makedirs(dtDir)

# -- set output file name    
contsALFile=dtDir+"/%s_mean_TopoCheck_conts_age_grid_AL.dat" %(model)
oceanALFile=dtDir+"/%s_mean_TopoCheck_ocean_age_grid_AL.dat" %(model)
oceanAreaFile=dtDir+"/%s_OceanArea.dat" %(model)


CF=open(contsALFile, 'w')
OF=open(oceanALFile, 'w')
AreaF=open(oceanAreaFile, 'w')

In [11]:
# -- path to model time file
timeFile="/Users/ajy321/PhD_work/SeaLevel/SeaLevel_MS/GPlatesFiles/C1/TotalTopography/C1TT.timese"

# -- read time file
f=open(timeFile)

Age_List=[]
Step_List=[]
for line in f:
    parts = line.split() # split line into parts
    if len(parts) > 1:   # if at least 2 parts/columns
        Step_List.append(parts[0])
        Age_List.append(parts[1])
# Age_List = Age_List[20:]
# Step_List = Step_List[20:]
# print (Step_List)
# print (Age_List)

In [12]:
# Iterate over time steps
# reconstruction_times = np.arange(min_time, max_time, time_step)
# for reconstruction_time in reconstruction_times:

# -- create time file loop
for i in range(len(Age_List)):
    reconstruction_time=int(Age_List[i])    
    step=int(Step_List[i])    
    print ("Age is: ",round(reconstruction_time, 1), "Ma")
    print ("Step is: ", (step))

    #path to in dt grid
    DTgrid=DTgridDir+'%sDT_MantleFrameGrid%s.grd' %(model, reconstruction_time)

    #path to age grid
    AGEgrid=AGEgridDir+'agegrid_final_masked_%s.nc' %(reconstruction_time)
    
    #resample topo grd
    cmd = "grdsample %s -I1d -Gtopogrd_1by1.nc -Rg" %(DTgrid)
    print cmd
    os.system(cmd)
    
    # Calculate ocean area
    cmd = "gmt grdvolume %s -C0 | awk '{print $2}'" %(AGEgrid)
    print cmd
    os.system(cmd)
    
    # write ocean area to output file
    OceanArea=(float(os.popen(cmd).read()))
    AreaF.write("%d %d\n" % (reconstruction_time, OceanArea))
    print (reconstruction_time, OceanArea)

    # Create a continental and oceanic mask from the age grids
    cmd= 'grdmath %s ISNAN = tmp2.nc' %(AGEgrid)
    print cmd
    os.system(cmd)
    
    cmd= 'grdclip -V tmp2.nc -Gtmp3.nc -Sa0/NaN'
    print cmd
    os.system(cmd)

    cmd= 'grdsample tmp3.nc -I1d -Gconts_nans.nc'
    print cmd
    os.system(cmd)

    cmd= 'grdclip -V tmp2.nc -Gtmp5.nc -Sb1/NaN'
    print cmd
    os.system(cmd)

    cmd= 'grdsample tmp5.nc -I1d -Gocean_nans.nc'
    print cmd
    os.system(cmd)
    
    # use mask to extract DT values from grid
    cmd = 'grdmath -V topogrd_1by1.nc 1 MUL = tmp1.nc'
    print cmd
    os.system(cmd)
    
    cmd = 'grdmath -V tmp1.nc conts_nans.nc OR = oceans_AL_%s.nc' %(reconstruction_time)
    print cmd
    os.system(cmd)
    
    cmd = 'grdmath -V tmp1.nc ocean_nans.nc OR = conts_AL_%s.nc' %(reconstruction_time)
    print cmd
    os.system(cmd)
    
    #calculate are weighted mean oceanic DT
    cmd = 'grdmath -V oceans_AL_%s.nc Y COSD MUL MEAN = tmp_wz.nc' %(reconstruction_time)
    print cmd
    os.system(cmd)

    cmd = 'grdmath -V oceans_AL_%s.nc 0 MUL Y COSD ADD MEAN = tmp_w.nc' %(reconstruction_time)
    print cmd
    os.system(cmd)

    cmd = 'grdmath -V tmp_wz.nc tmp_w.nc DIV = tmp_z.nc'
    print cmd
    os.system(cmd)

    cmd = "grdinfo -L2 tmp_z.nc | grep mean | awk '{print $3}'"
    print cmd
    os.system(cmd)
    
    meanOceans=(float(os.popen(cmd).read()))
    print (reconstruction_time, meanOceans)

    # write results to output file
    OF.write("%d %d\n" % (reconstruction_time, meanOceans))
    
    #calculate are weighted mean continental DT
    cmd = 'grdmath -V conts_AL_%s.nc Y COSD MUL MEAN = tmp_wz2.nc' %(reconstruction_time)
    print cmd
    os.system(cmd)
    
    cmd = 'grdmath -V conts_AL_%s.nc 0 MUL Y COSD ADD MEAN = tmp_w2.nc' %(reconstruction_time)
    print cmd
    os.system(cmd)
    
    cmd = 'grdmath -V tmp_wz2.nc tmp_w2.nc DIV = tmp_z2.nc'
    print cmd
    os.system(cmd)
    
    cmd = "grdinfo -L2 tmp_z2.nc | grep mean | awk '{print $3}'"
    print cmd
    os.system(cmd)
    
    meanConts=(float(os.popen(cmd).read()))
    print (reconstruction_time, meanConts)

    # write results to output file
    CF.write("%d %d\n" % (reconstruction_time, meanConts))
    
    # tidy up
    cmd = 'rm tmp*.nc conts_nans.nc ocean_nans.nc topogrd_1by1.nc'
    print cmd
    os.system(cmd)
    
OF.close()   
CF.close() 
AreaF.close()

('Age is: ', 560.0, 'Ma')
('Step is: ', 16015)
grdsample /Users/ajy321/PhD_work/SeaLevel/SeaLevel_MS/GPlatesFiles/C3/DynamicTopography/C3DT_MantleFrameGrid560.grd -I1d -Gtopogrd_1by1.nc -Rg
gmt grdvolume /Users/ajy321/PhD_work/SeaLevel/EPSL_r3_calcs_05072021/AgeGrids/Mask/agegrid_final_masked_560.nc -C0 | awk '{print $2}'
(560, 354873996929000.0)
grdmath /Users/ajy321/PhD_work/SeaLevel/EPSL_r3_calcs_05072021/AgeGrids/Mask/agegrid_final_masked_560.nc ISNAN = tmp2.nc
grdclip -V tmp2.nc -Gtmp3.nc -Sa0/NaN
grdsample tmp3.nc -I1d -Gconts_nans.nc
grdclip -V tmp2.nc -Gtmp5.nc -Sb1/NaN
grdsample tmp5.nc -I1d -Gocean_nans.nc
grdmath -V topogrd_1by1.nc 1 MUL = tmp1.nc
grdmath -V tmp1.nc conts_nans.nc OR = oceans_AL_560.nc
grdmath -V tmp1.nc ocean_nans.nc OR = conts_AL_560.nc
grdmath -V oceans_AL_560.nc Y COSD MUL MEAN = tmp_wz.nc
grdmath -V oceans_AL_560.nc 0 MUL Y COSD ADD MEAN = tmp_w.nc
grdmath -V tmp_wz.nc tmp_w.nc DIV = tmp_z.nc
grdinfo -L2 tmp_z.nc | grep mean | awk '{print $3}'
(560, 111.

grdmath -V tmp1.nc conts_nans.nc OR = oceans_AL_440.nc
grdmath -V tmp1.nc ocean_nans.nc OR = conts_AL_440.nc
grdmath -V oceans_AL_440.nc Y COSD MUL MEAN = tmp_wz.nc
grdmath -V oceans_AL_440.nc 0 MUL Y COSD ADD MEAN = tmp_w.nc
grdmath -V tmp_wz.nc tmp_w.nc DIV = tmp_z.nc
grdinfo -L2 tmp_z.nc | grep mean | awk '{print $3}'
(440, 180.227401429)
grdmath -V conts_AL_440.nc Y COSD MUL MEAN = tmp_wz2.nc
grdmath -V conts_AL_440.nc 0 MUL Y COSD ADD MEAN = tmp_w2.nc
grdmath -V tmp_wz2.nc tmp_w2.nc DIV = tmp_z2.nc
grdinfo -L2 tmp_z2.nc | grep mean | awk '{print $3}'
(440, -443.62774457)
rm tmp*.nc conts_nans.nc ocean_nans.nc topogrd_1by1.nc
('Age is: ', 420.0, 'Ma')
('Step is: ', 22785)
grdsample /Users/ajy321/PhD_work/SeaLevel/SeaLevel_MS/GPlatesFiles/C3/DynamicTopography/C3DT_MantleFrameGrid420.grd -I1d -Gtopogrd_1by1.nc -Rg
gmt grdvolume /Users/ajy321/PhD_work/SeaLevel/EPSL_r3_calcs_05072021/AgeGrids/Mask/agegrid_final_masked_420.nc -C0 | awk '{print $2}'
(420, 358448799650000.0)
grdmath /User

grdmath -V tmp_wz2.nc tmp_w2.nc DIV = tmp_z2.nc
grdinfo -L2 tmp_z2.nc | grep mean | awk '{print $3}'
(320, -188.567397805)
rm tmp*.nc conts_nans.nc ocean_nans.nc topogrd_1by1.nc
('Age is: ', 300.0, 'Ma')
('Step is: ', 27968)
grdsample /Users/ajy321/PhD_work/SeaLevel/SeaLevel_MS/GPlatesFiles/C3/DynamicTopography/C3DT_MantleFrameGrid300.grd -I1d -Gtopogrd_1by1.nc -Rg
gmt grdvolume /Users/ajy321/PhD_work/SeaLevel/EPSL_r3_calcs_05072021/AgeGrids/Mask/agegrid_final_masked_300.nc -C0 | awk '{print $2}'
(300, 320021683421000.0)
grdmath /Users/ajy321/PhD_work/SeaLevel/EPSL_r3_calcs_05072021/AgeGrids/Mask/agegrid_final_masked_300.nc ISNAN = tmp2.nc
grdclip -V tmp2.nc -Gtmp3.nc -Sa0/NaN
grdsample tmp3.nc -I1d -Gconts_nans.nc
grdclip -V tmp2.nc -Gtmp5.nc -Sb1/NaN
grdsample tmp5.nc -I1d -Gocean_nans.nc
grdmath -V topogrd_1by1.nc 1 MUL = tmp1.nc
grdmath -V tmp1.nc conts_nans.nc OR = oceans_AL_300.nc
grdmath -V tmp1.nc ocean_nans.nc OR = conts_AL_300.nc
grdmath -V oceans_AL_300.nc Y COSD MUL MEAN = 

grdclip -V tmp2.nc -Gtmp3.nc -Sa0/NaN
grdsample tmp3.nc -I1d -Gconts_nans.nc
grdclip -V tmp2.nc -Gtmp5.nc -Sb1/NaN
grdsample tmp5.nc -I1d -Gocean_nans.nc
grdmath -V topogrd_1by1.nc 1 MUL = tmp1.nc
grdmath -V tmp1.nc conts_nans.nc OR = oceans_AL_180.nc
grdmath -V tmp1.nc ocean_nans.nc OR = conts_AL_180.nc
grdmath -V oceans_AL_180.nc Y COSD MUL MEAN = tmp_wz.nc
grdmath -V oceans_AL_180.nc 0 MUL Y COSD ADD MEAN = tmp_w.nc
grdmath -V tmp_wz.nc tmp_w.nc DIV = tmp_z.nc
grdinfo -L2 tmp_z.nc | grep mean | awk '{print $3}'
(180, 281.828521851)
grdmath -V conts_AL_180.nc Y COSD MUL MEAN = tmp_wz2.nc
grdmath -V conts_AL_180.nc 0 MUL Y COSD ADD MEAN = tmp_w2.nc
grdmath -V tmp_wz2.nc tmp_w2.nc DIV = tmp_z2.nc
grdinfo -L2 tmp_z2.nc | grep mean | awk '{print $3}'
(180, -8.14523697195)
rm tmp*.nc conts_nans.nc ocean_nans.nc topogrd_1by1.nc
('Age is: ', 160.0, 'Ma')
('Step is: ', 33368)
grdsample /Users/ajy321/PhD_work/SeaLevel/SeaLevel_MS/GPlatesFiles/C3/DynamicTopography/C3DT_MantleFrameGrid160.grd -

grdinfo -L2 tmp_z.nc | grep mean | awk '{print $3}'
(60, 230.082686436)
grdmath -V conts_AL_60.nc Y COSD MUL MEAN = tmp_wz2.nc
grdmath -V conts_AL_60.nc 0 MUL Y COSD ADD MEAN = tmp_w2.nc
grdmath -V tmp_wz2.nc tmp_w2.nc DIV = tmp_z2.nc
grdinfo -L2 tmp_z2.nc | grep mean | awk '{print $3}'
(60, -235.062469969)
rm tmp*.nc conts_nans.nc ocean_nans.nc topogrd_1by1.nc
('Age is: ', 40.0, 'Ma')
('Step is: ', 38472)
grdsample /Users/ajy321/PhD_work/SeaLevel/SeaLevel_MS/GPlatesFiles/C3/DynamicTopography/C3DT_MantleFrameGrid40.grd -I1d -Gtopogrd_1by1.nc -Rg
gmt grdvolume /Users/ajy321/PhD_work/SeaLevel/EPSL_r3_calcs_05072021/AgeGrids/Mask/agegrid_final_masked_40.nc -C0 | awk '{print $2}'
(40, 278350730652000.0)
grdmath /Users/ajy321/PhD_work/SeaLevel/EPSL_r3_calcs_05072021/AgeGrids/Mask/agegrid_final_masked_40.nc ISNAN = tmp2.nc
grdclip -V tmp2.nc -Gtmp3.nc -Sa0/NaN
grdsample tmp3.nc -I1d -Gconts_nans.nc
grdclip -V tmp2.nc -Gtmp5.nc -Sb1/NaN
grdsample tmp5.nc -I1d -Gocean_nans.nc
grdmath -V topogr